In [1]:
import pandas as pd

In [2]:
import pyarrow.parquet as pq

In [3]:
file_path = "data/raw/removals-latest.parquet"

In [4]:
# Create a Parquet file object
parquet_file = pq.ParquetFile(file_path)

In [5]:
for batch in parquet_file.iter_batches(batch_size=10000):
    # Convert the batch to a pandas DataFrame
    df = batch.to_pandas()
    # Perform your duplicate checks here

In [6]:
# df = pd.read_excel(file_path)

In [13]:
df.head(2)

,departed_date,port_of_departure,departure_country,docket_aor,apprehension_state,apprehension_county,case_status,gender,birth_country,citizenship_country,...,final_order_date,final_charge_code,final_charge_section_code,prior_deport_yes_no,latest_person_departed_date,unique_identifier,duplicate_likely,file_original,sheet_original,row_original
0,2024-08-01,"ALEXANDRIA INTERNATIONAL AIRPORT, POE",HONDURAS,New Orleans Area of Responsibility,NaN,NaN,8-Excluded/Removed - Inadmissibility,Male,HONDURAS,HONDURAS,...,2018-01-16,I6A,212a6Ai,NO,2024-08-01,fc75905f47abfe9bd89cd25505a7588f0c7de15b,0.0,2025-ICLI-00019_2024-ICFO-39357_ICE Removals_L...,Removals,520008
1,2023-10-31,"MCALLEN, TX, POE",HONDURAS,Harlingen Area of Responsibility,NaN,NaN,8-Excluded/Removed - Inadmissibility,Female,HONDURAS,HONDURAS,...,2023-10-27,I7A1,212a7AiI,NO,2023-10-31,fc759dc6e403bfc54b8006f76ebe8f0dfa3460a1,0.0,2025-ICLI-00019_2024-ICFO-39357_ICE Removals_L...,Removals,520009


In [9]:
df.info()


<class 'pandas.DataFrame'>
RangeIndex: 8441 entries, 0 to 8440
Data columns (total 41 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   departed_date                       8441 non-null   datetime64[us]
 1   port_of_departure                   8441 non-null   str           
 2   departure_country                   8441 non-null   str           
 3   docket_aor                          8441 non-null   str           
 4   apprehension_state                  2074 non-null   str           
 5   apprehension_county                 0 non-null      str           
 6   case_status                         8441 non-null   str           
 7   gender                              8441 non-null   str           
 8   birth_country                       8441 non-null   str           
 9   citizenship_country                 8441 non-null   str           
 10  birth_year                         

In [14]:
# df['anonymized_identifer'].is_unique
df['unique_identifier'].is_unique

False

In [11]:
# This counts how many times each ID appears and shows the top offenders
duplicate_counts = df['anonymized_identifer'].value_counts()
print(duplicate_counts[duplicate_counts > 1])

KeyError: 'anonymized_identifer'

In [ ]:
# Define the ID you found with 29 occurrences
target_id = "4326c42c2d4bb3ac4c97f693dfc3c3b7145b933c"

# Filter the dataframe
individual_audit = df[df['anonymized_identifer'] == target_id]

# Sort by date to see the timeline of removals
individual_audit = individual_audit.sort_values('departure_date')

# Display the key personal columns to check for consistency
columns_to_check = [
    'anonymized_identifer', 'birth_year', 'gender', 
    'birth_country', 'departure_date', 'port_of_departure'
]

individual_audit[columns_to_check]

In [ ]:
# If the result is 1, the attribute is consistent across all 29 rows
print(f"Unique birth years: {individual_audit['birth_year'].nunique()}")
print(f"Unique genders: {individual_audit['gender'].nunique()}")
print(f"Unique birth countries: {individual_audit['birth_country'].nunique()}")

In [ ]:
# Create a sequence number for each individual based on their departure date
df['removal_sequence'] = df.sort_values('departure_date').groupby('anonymized_identifer').cumcount() + 1

# Check our target individual again to see the new column
df[df['anonymized_identifer'] == '4326c42c2d4bb3ac4c97f693dfc3c3b7145b933c'].sort_values('departure_date')

In [ ]:
import matplotlib.pyplot as plt

# Count how many removals each unique ID has
counts = df.groupby('anonymized_identifer').size()

# Plot the distribution
plt.figure(figsize=(10, 6))
counts.value_counts().sort_index().head(29).plot(kind='bar', color='red')
plt.title('Distribution of Repeat Removals (First 10 counts)')
plt.xlabel('Number of times an individual was removed')
plt.ylabel('Number of Individuals')
plt.yscale('log') # Log scale because most people are only removed once
plt.show()